# A Gentle Introduction to RAG Applications

This notebook creates a simple RAG (Retrieval-Augmented Generation) system to answer questions from a PDF document using an open-source model.

In [ ]:
PDF_FILE = "paul.pdf"

# We'll be using Llama 3.2 8B for this example.
MODEL = "llama3.2"

## Loading the PDF document

Let's start by loading the PDF document and breaking it down into separate pages.

<img src='images/documents.png' width="1000">

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(PDF_FILE)
pages = loader.load()

print(f"Number of pages: {len(pages)}")
print(f"Length of a page: {len(pages[1].page_content)}")
print("Content of a page:", pages[1].page_content)

Number of pages: 9
Length of a page: 3272
Content of a page: 10% a week. And while 110 may not seem much better than 100,
if you keep growing at 10% a week you'll be surprised how big
the numbers get. After a year you'll have 14,000 users, and after
2 years you'll have 2 million.
You'll be doing different things when you're acquiring users a
thousand at a time, and growth has to slow down eventually. But
if the market exists you can usually start by recruiting users
manually and then gradually switch to less manual methods. [3]
Airbnb is a classic example of this technique. Marketplaces are so
hard to get rolling that you should expect to take heroic measures
at first. In Airbnb's case, these consisted of going door to door in
New York, recruiting new users and helping existing ones improve
their listings. When I remember the Airbnbs during YC, I picture
them with rolly bags, because when they showed up for tuesday
dinners they'd always just flown back from somewhere.
Fragile
Airbnb no

## Splitting the pages in chunks

Pages are too long, so let's split pages into different chunks.

<img src='images/splitter.png' width="1000">


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)

chunks = splitter.split_documents(pages)
print(f"Number of chunks: {len(chunks)}")
print(f"Length of a chunk: {len(chunks[1].page_content)}")
print("Content of a chunk:", chunks[1].page_content)


Number of chunks: 23
Length of a chunk: 1236
Content of a chunk: took better advantage of it than Stripe. At YC we use the term
"Collison installation" for the technique they invented. More
diffident founders ask "Will you try our beta?" and if the answer is
yes, they say "Great, we'll send you a link." But the Collison
brothers weren't going to wait. When anyone agreed to try Stripe
they'd say "Right then, give me your laptop" and set them up on
the spot.
There are two reasons founders resist going out and recruiting
users individually. One is a combination of shyness and laziness.
They'd rather sit at home writing code than go out and talk to a
bunch of strangers and probably be rejected by most of them.
But for a startup to succeed, at least one founder (usually the
CEO) will have to spend a lot of time on sales and marketing. [2]
The other reason founders ignore this path is that the absolute
numbers seem so small at first. This can't be how the big, famous
startups got started, th

## Storing the chunks in a vector store

We can now generate embeddings for every chunk and store them in a vector store.

<img src='images/vectorstore.png' width="1000">


In [ ]:
# from langchain_community.vectorstores import FAISS
# from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
# from langchain_openai import OpenAIEmbeddings # Ou qualquer outro modelo de embedding
from langchain_ollama import OllamaEmbeddings
import os

persist_directory = "chroma_db"

embeddings = OllamaEmbeddings(model=MODEL)
# vectorstore = FAISS.from_documents(chunks, embeddings)

if not os.path.exists(persist_directory):
    print(f"Criando nova vector store em '{persist_directory}'...")
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory=persist_directory  # Importante: especifica onde salvar
    )
    print("Vector store criada e persistida.")
else:
    print(f"O diretório '{persist_directory}' já existe. Pulando a criação e indo para o carregamento.")
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings  # Importante: use a mesma função de embedding
    )
    print(f"Vector store carregada de '{persist_directory}'.")




Criando nova vector store em 'chroma_db_elefantes'...
Vector store criada e persistida.


## Setting up a retriever

We can use a retriever to find chunks in the vector store that are similar to a supplied question.

<img src='images/retriever.png' width="1000">



In [ ]:
retriever = vectorstore.as_retriever()
retriever.invoke("What can you get away with when you only have a small number of users?")

[Document(metadata={'page_label': '3', 'page': 2, 'producer': 'macOS Version 14.5 (Build 23F79) Quartz PDFContext, AppendMode 1.1', 'creator': 'Chrome', 'title': 'Do Things that Don_t Scale', 'total_pages': 9, 'creationdate': "D:20240806150448Z00'00'", 'source': 'paul.pdf', 'moddate': "D:20240806150500Z00'00'"}, page_content="in charge of their narrow domain of building things, rather than\nrunning the whole show. You can be ornery when you're Scotty,\nbut not when you're Kirk.\nAnother reason founders don't focus enough on individual\ncustomers is that they worry it won't scale. But when founders of\nlarval startups worry about this, I point out that in their current\nstate they have nothing to lose. Maybe if they go out of their way\nto make existing users super happy, they'll one day have too\nmany to do so much for. That would be a great problem to have.\nSee if you can make it happen. And incidentally, when it does,\nyou'll find that delighting customers scales better than you\nex

## Configuring the model

We'll be using Ollama to load the local model in memory. After creating the model, we can invoke it with a question to get the response back.

<img src='images/model.png' width="1000">

In [11]:
from langchain_ollama import ChatOllama

model = ChatOllama(model=MODEL, temperature=0)
model.invoke("Who is the president of the United States?")

AIMessage(content='As of my last update in April 2023, Joe Biden is the President of the United States. He took office on January 20, 2021, succeeding Donald Trump as the 46th President of the United States. Please note that this information might change over time due to elections or other political developments.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-05-08T13:50:11.878604Z', 'done': True, 'done_reason': 'stop', 'total_duration': 18528667250, 'load_duration': 71832084, 'prompt_eval_count': 19, 'prompt_eval_duration': 9629799375, 'eval_count': 65, 'eval_duration': 8810673917, 'model_name': 'llama3.1'}, id='run--5bdf3abc-fa8d-4806-9639-9de79b44c205-0', usage_metadata={'input_tokens': 19, 'output_tokens': 65, 'total_tokens': 84})

## Parsing the model's response

The response from the model is an `AIMessage` instance containing the answer. We can extract the text answer by using the appropriate output parser. We can connect the model and the parser using a chain.

<img src='images/parser.png' width="1000">


In [12]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
print(chain.invoke("Who is the president of the United States?"))

As of my last update in April 2023, Joe Biden is the President of the United States. He took office on January 20, 2021, succeeding Donald Trump as the 46th President of the United States. Please note that this information might change over time due to elections or other political developments.


## Setting up a prompt

In addition to the question we want to ask, we also want to provide the model with the context from the PDF file. We can use a prompt template to define and reuse the prompt we'll use with the model.


<img src='images/prompt.png' width="1000">

In [13]:
from langchain.prompts import PromptTemplate

template = """
You are an assistant that provides answers to questions based on
a given context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


You are an assistant that provides answers to questions based on
a given context. 

Answer the question based on the context. If you can't answer the
question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: Here is some context

Question: Here is a question



## Adding the prompt to the chain

We can now chain the prompt with the model and the parser.

<img src='images/chain1.png' width="1000">

In [14]:
chain = prompt | model | parser

chain.invoke({
    "context": "Anna's sister is Susan", 
    "question": "Who is Susan's sister?"
})


'Anna.'

## Adding the retriever to the chain

Finally, we can connect the retriever to the chain to get the context from the vector store.

<img src='images/chain2.png' width="1000">

In [15]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

## Using the chain to answer questions

Finally, we can use the chain to ask questions that will be answered using the PDF document.

In [16]:
questions = [
    "What can you get away with when you only have a small number of users?",
    "What's the most common unscalable thing founders have to do at the start?",
    "What's one of the biggest things inexperienced founders and investors get wrong about startups?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print("*************************\n")

Question: What can you get away with when you only have a small number of users?
Answer: You can provide a level of service no big company can.
*************************

Question: What's the most common unscalable thing founders have to do at the start?
Answer: Delighting customers.
*************************

Question: What's one of the biggest things inexperienced founders and investors get wrong about startups?
Answer: They underestimate the potential of their startup.
*************************

